In [1]:
import os
import shutil

import torch
import torch.utils.data
# import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import argparse
import re
import pickle
from helpers import makedir
import protopnet
import push
import prune
import numpy as np
import save
from log import create_logger
from preprocess import mean, std, preprocess_input_function

In [2]:
import copy
import glob
import os
import os.path as osp
import shutil
import sys
from pathlib import Path
from types import SimpleNamespace

import cv2

# solve potential deadlock https://github.com/pytorch/pytorch/issues/1355
cv2.setNumThreads(0)

import hydra
import torch.backends.cudnn as cudnn
import wandb
from omegaconf import DictConfig
from torch.utils.data import DataLoader

from unsup_datasets.lit_dataset import LitDataset
from utils.utils import seed_worker
import resource
rlimit = resource.getrlimit(resource.RLIMIT_NOFILE)
resource.setrlimit(resource.RLIMIT_NOFILE, (4096, rlimit[1]))
import matplotlib.pyplot as plt

# from hydra import compose, initialize
# from omegaconf import OmegaConf

# initialize(config_path="unsup_configs", job_name="test_app")

In [3]:
# cfg = compose(config_name="config.yaml")
# test = OmegaConf.to_yaml(cfg)

# # print(os.getcwd())
# cfg = dict(cfg)
# # if cfg['dataset_name'] is not None:
# #     cfg['dataset'] = cfg['dataset_name']
# num_classes = 20
# cfg['proto_num_classes'] = num_classes
# # Entered my username
# wandb.init(project='unsup-parts', entity='kadziolka-marcin', mode='disabled', allow_val_change=True)
# wandb.config.update(cfg, allow_val_change=True)
# args = wandb.config
# cudnn.enabled = True
num_classes = 20
# if args.exp_name is not None:
#     api = wandb.Api()
#     run = api.run(path=f'wandb_userid/unsup-parts/{wandb.run.id}')
#     run.name = f'{args.exp_name}-{run.name}'
#     run.save()

# print("---------------------------------------")
# print(f"Arguments received: ")
# print("---------------------------------------")
# for k, v in sorted(args.items()):
#     print(f"{k:25}: {v}")
# print("---------------------------------------")


# OURS
# SETTINGS FILE

# our default architecture
# it is used in ppnet class to download pretrained resnet34
base_architecture = 'resnet34'
img_size = 224

# 4 prototypes per 1 class, 20 classes == 80 prototypes
prototype_shape = (80, 128, 1, 1)

prototype_activation_function = 'log'
add_on_layers_type = 'regular'

train_batch_size = 20
test_batch_size = 20
train_push_batch_size = 20

from datasets.PartsDataLoader import PDL
root = './'

normalize = transforms.Normalize(mean=mean,
                                 std=std)

trans = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    normalize # very important
])

trans_push = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    # don't normalize push loader!
])


ds_train = PDL(root, train=True, transform=trans, num_classes=num_classes, images=["images", "rotated", "sheared", "skewed"])
print(len(ds_train))
ds_train_push = PDL(root, train=True, transform=trans_push, num_classes=num_classes, images=["images", "rotated", "sheared", "skewed"])
print(len(ds_train_push))
ds_test = PDL(root, train=False, transform=trans, num_classes=num_classes, images=['images'])#['cropped'])
print(len(ds_test))

train_push_loader = torch.utils.data.DataLoader(ds_train_push, batch_size=train_push_batch_size, shuffle=True, num_workers=2)
train_loader = torch.utils.data.DataLoader(ds_train, batch_size=train_batch_size, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(ds_test, batch_size=test_batch_size, shuffle=True, num_workers=2)

2372
2372
512


In [4]:
# for data, label in train_loader:
#     img = data['IMG'][0]
#     parts = data['PART_0'][0], data['PART_1'][0], data['PART_2'][0], data['PART_3'][0]
#     # print images using matplotlib
    
#     r_index = 0
#     f, axarr = plt.subplots(1, 5, figsize=(20,15))
#     axarr[0].imshow(np.transpose(img.numpy(), (1, 2, 0)))
#     axarr[1].imshow(np.transpose(parts[0].numpy(), (1, 2, 0)), cmap="gray")
#     axarr[2].imshow(np.transpose(parts[1].numpy(), (1, 2, 0)), cmap="gray")
#     axarr[3].imshow(np.transpose(parts[2].numpy(), (1, 2, 0)), cmap="gray")
#     axarr[4].imshow(np.transpose(parts[3].numpy(), (1, 2, 0)), cmap="gray");
#     plt.title(label[0])
#     plt.show()
    
#     break

In [5]:
# ours_classes = {}
# needed_classes = list(range(20))
# print(needed_classes)
# for data, labels in test_loader:
#     image = data['IMG']
#     for i in range(len(labels)):
#         if labels[i] in needed_classes:
#             ours_classes[labels[i]] = image[i]
#             needed_classes.remove(labels[i])
# print(ours_classes.keys())
# print(needed_classes)

In [6]:
# fig, axs = plt.subplots(4, 5, figsize=(10, 10))
# for idx, (label, image) in enumerate(sorted(ours_classes.items())):
#     i = idx//5
#     j = idx%5
#     axs[i, j].imshow(np.transpose(image, (1, 2, 0)))
#     axs[i, j].set_title(label.numpy())
#     axs[i, j].axis('off')
# plt.show()

In [7]:
from datetime import datetime
now = datetime.now()
dt_string = now.strftime("%d_%m_%Y_%H_%M_%S")
model_dir = f"PDL_ProtoPNet_base_arch_{base_architecture}_n_classes_{num_classes}_prot_shape_{prototype_shape}_{dt_string}"
model_dir = "TESTING"
makedir(model_dir)

In [8]:
log, logclose = create_logger(log_filename=os.path.join(model_dir, 'train.log'))
img_dir = os.path.join(model_dir, 'img')
makedir(img_dir)
weight_matrix_filename = 'outputL_weights'
prototype_img_filename_prefix = 'prototype-img'
prototype_self_act_filename_prefix = 'prototype-self-act'
proto_bound_boxes_filename_prefix = 'bb'

In [9]:
log('training set size: {0}'.format(len(train_loader.dataset)))
# log('push set size: {0}'.format(len(train_push_loader.dataset)))
log('test set size: {0}'.format(len(test_loader.dataset)))
# log('train batch size: {0}'.format(train_batch_size))

# print(len(train_loader.dataset))
# print(len(train_push_loader.dataset))
# print(len(test_loader.dataset))
# print(train_batch_size)

# construct the model
ppnet = protopnet.construct_PPNet(base_architecture=base_architecture,
                              pretrained=True, img_size=img_size,
                              prototype_shape=prototype_shape,
                              num_classes=num_classes,
                              prototype_activation_function=prototype_activation_function,
                              add_on_layers_type=add_on_layers_type)

ppnet = ppnet.cuda()
ppnet_multi = torch.nn.DataParallel(ppnet)
class_specific = True

training set size: 2372
test set size: 512


In [10]:
# define optimizer

joint_optimizer_lrs = {'features': 1e-4,
                       'add_on_layers': 3e-3,
                       'prototype_vectors': 3e-3}
joint_lr_step_size = 5

warm_optimizer_lrs = {'add_on_layers': 3e-3,
                      'prototype_vectors': 3e-3}

last_layer_optimizer_lr = 1e-4

coefs = {
    'crs_ent': 1,
    'clst': 0.8,
    'sep': -0.08,
    'l1': 1e-4,
}

joint_optimizer_specs = \
[{'params': ppnet.features.parameters(), 'lr': joint_optimizer_lrs['features'], 'weight_decay': 1e-3}, # bias are now also being regularized
 {'params': ppnet.add_on_layers.parameters(), 'lr': joint_optimizer_lrs['add_on_layers'], 'weight_decay': 1e-3},
 {'params': ppnet.prototype_vectors, 'lr': joint_optimizer_lrs['prototype_vectors']},
]
joint_optimizer = torch.optim.Adam(joint_optimizer_specs)
joint_lr_scheduler = torch.optim.lr_scheduler.StepLR(joint_optimizer, step_size=joint_lr_step_size, gamma=0.1)


warm_optimizer_specs = \
[{'params': ppnet.add_on_layers.parameters(), 'lr': warm_optimizer_lrs['add_on_layers'], 'weight_decay': 1e-3},
 {'params': ppnet.prototype_vectors, 'lr': warm_optimizer_lrs['prototype_vectors']},
]
warm_optimizer = torch.optim.Adam(warm_optimizer_specs)


last_layer_optimizer_specs = [{'params': ppnet.last_layer.parameters(), 'lr': last_layer_optimizer_lr}]
last_layer_optimizer = torch.optim.Adam(last_layer_optimizer_specs)

# weighting of different training losses



# number of training epochs, number of warm epochs, push start epoch, push epochs
num_train_epochs = 100
num_warm_epochs = 5
push_start = 10
push_epochs = [i for i in range(num_train_epochs) if i % 10 == 0]

In [11]:
def load_model(model_dir=None, model_name=None, ppnet=None, joint_optimizer=None, warm_optimizer=None, last_layer_optimizer=None):
    # none everywhere as to not load anything that jupyter holds in memory
    model_path = model_dir + '/' + model_name
    joint_optimizer_path = model_dir + '/joint_optimizer.pth'
    warm_optimizer_path = model_dir + '/warm_optimizer.pth'
    last_layer_optimizer_path = model_dir + '/last_layer_optimizer.pth'
    variables_path = model_dir + '/variables.pkl'


    # WARNING: THEY SAVE WHOLE MODEL, I SAVE DICT STATE
    ppnet.load_state_dict(torch.load(model_path))
    
    # Load the optimizer states
    joint_optimizer.load_state_dict(torch.load(joint_optimizer_path))
    warm_optimizer.load_state_dict(torch.load(warm_optimizer_path))
    last_layer_optimizer.load_state_dict(torch.load(last_layer_optimizer_path))
    
    # It probably isn't that useful, so they are not returned
    with open(variables_path, 'rb') as f:
        variables = pickle.load(f)
    coefs = variables['coefs']
    num_train_epochs = variables['num_train_epochs']
    num_warm_epochs = variables['num_warm_epochs']
    push_start = variables['push_start']
    push_epochs = variables['push_epochs']
    class_specific = variables['class_specific']


    
    # Load other variables as needed
    log(f"Model and optimizers loaded from {model_dir}")
    log(f"Model: {model_path}")
    log(f"Joint optimizer: {joint_optimizer_path}")
    log(f"Warm optimizer: {warm_optimizer_path}")
    log(f"Last layer_optimizer: {last_layer_optimizer_path}")
    # log(f"Variables: {variables_path}")

In [12]:
load_model_dir = 'ProtoPNet_base_arch_resnet34_n_classes_20_prot_shape_(80, 128, 1, 1)_28_05_2023_15_15_19'
load_model_name = 'ProtoPNet_base_arch_resnet34_n_classes_20_prot_shape_(80, 128, 1, 1)_28_05_2023_15_15_19_0.8019417475728156.pth'
load_model(model_dir=load_model_dir, model_name=load_model_name, ppnet=ppnet, joint_optimizer=joint_optimizer, warm_optimizer=warm_optimizer, last_layer_optimizer=last_layer_optimizer)

Model and optimizers loaded from ProtoPNet_base_arch_resnet34_n_classes_20_prot_shape_(80, 128, 1, 1)_28_05_2023_15_15_19
Model: ProtoPNet_base_arch_resnet34_n_classes_20_prot_shape_(80, 128, 1, 1)_28_05_2023_15_15_19/ProtoPNet_base_arch_resnet34_n_classes_20_prot_shape_(80, 128, 1, 1)_28_05_2023_15_15_19_0.8019417475728156.pth
Joint optimizer: ProtoPNet_base_arch_resnet34_n_classes_20_prot_shape_(80, 128, 1, 1)_28_05_2023_15_15_19/joint_optimizer.pth
Warm optimizer: ProtoPNet_base_arch_resnet34_n_classes_20_prot_shape_(80, 128, 1, 1)_28_05_2023_15_15_19/warm_optimizer.pth
Last layer_optimizer: ProtoPNet_base_arch_resnet34_n_classes_20_prot_shape_(80, 128, 1, 1)_28_05_2023_15_15_19/last_layer_optimizer.pth


In [13]:
# train the model
import unsup_train_and_test as utnt
target_acc = 0.90
log('start training')
import copy
for epoch in range(num_train_epochs):
    log('epoch: \t{0}'.format(epoch))

    if epoch < num_warm_epochs:
        utnt.warm_only(model=ppnet_multi, log=log)
        _ = utnt.train(model=ppnet_multi, dataloader=train_loader, optimizer=warm_optimizer,
                      class_specific=class_specific, coefs=coefs, log=log)
    else:
        utnt.joint(model=ppnet_multi, log=log)
        joint_lr_scheduler.step()
        _ = utnt.train(model=ppnet_multi, dataloader=train_loader, optimizer=joint_optimizer,
                      class_specific=class_specific, coefs=coefs, log=log)

    accu = utnt.test(model=ppnet_multi, dataloader=test_loader,
                    class_specific=class_specific, log=log)
    save.save_model_w_condition(model=ppnet, model_dir=model_dir, model_name=str(epoch) + 'nopush', accu=accu,
                                target_accu=target_acc, log=log)

    if epoch >= push_start and epoch in push_epochs:
        push.push_prototypes(
            train_push_loader, # pytorch dataloader (must be unnormalized in [0,1])
            prototype_network_parallel=ppnet_multi, # pytorch network with prototype_vectors
            class_specific=class_specific,
            preprocess_input_function=preprocess_input_function, # normalize if needed
            prototype_layer_stride=1,
            root_dir_for_saving_prototypes=img_dir, # if not None, prototypes will be saved here
            epoch_number=epoch, # if not provided, prototypes saved previously will be overwritten
            prototype_img_filename_prefix=prototype_img_filename_prefix,
            prototype_self_act_filename_prefix=prototype_self_act_filename_prefix,
            proto_bound_boxes_filename_prefix=proto_bound_boxes_filename_prefix,
            save_prototype_class_identity=True,
            log=log)
        accu = utnt.test(model=ppnet_multi, dataloader=test_loader,
                        class_specific=class_specific, log=log)
        save.save_model_w_condition(model=ppnet, model_dir=model_dir, model_name=str(epoch) + 'push', accu=accu,
                                    target_accu=target_acc, log=log)

        if prototype_activation_function != 'linear':
            utnt.last_only(model=ppnet_multi, log=log)
            for i in range(20):
                log('iteration: \t{0}'.format(i))
                _ = utnt.train(model=ppnet_multi, dataloader=train_loader, optimizer=last_layer_optimizer,
                              class_specific=class_specific, coefs=coefs, log=log)
                accu = utnt.test(model=ppnet_multi, dataloader=test_loader,
                                class_specific=class_specific, log=log)
                save.save_model_w_condition(model=ppnet, model_dir=model_dir, model_name=str(epoch) + '_' + str(i) + 'push', accu=accu,
                                            target_accu=target_acc, log=log)
   
logclose()

start training
epoch: 	0
	warm
	train
	time: 	23.13093400001526
	cross ent: 	1.0265217165486151
	cluster: 	0.21048224803467258
	separation:	0.06734309064570673
	avg separation:	1.8611693111788326
	 accu: 		72.5548060708263%
	l1: 		806.6661376953125
	p dist pair: 	4.2457475662231445
	 total parts loss: 	697.19775390625
	test
	time: 	4.271228075027466
	cross ent: 	1.1262193402418723
	cluster: 	0.21753089960951072
	separation:	0.03157394157292751
	avg separation:	2.1248281643940854
	 accu: 		71.6796875%
	l1: 		806.6661376953125
	p dist pair: 	4.2457475662231445
	 total parts loss: 	545.233642578125
epoch: 	1
	warm
	train
	time: 	22.717811346054077
	cross ent: 	0.5845248729240995
	cluster: 	0.13705217985420667
	separation:	0.04622970137964277
	avg separation:	2.205498791542374
	 accu: 		83.13659359190557%
	l1: 		806.6661376953125
	p dist pair: 	4.495097637176514
	 total parts loss: 	632.024658203125
	test


KeyboardInterrupt: 

In [ ]:
# # SAVING MODEL TO TRAIN LATER

# # Save the model state
# model_path = model_dir + '/' + model_dir + '_' + str(accu) + '.pth'
# torch.save(ppnet.state_dict(), model_path)

# # Save the optimizer states
# joint_optimizer_path = model_dir + '/joint_optimizer.pth'
# torch.save(joint_optimizer.state_dict(), joint_optimizer_path)

# warm_optimizer_path = model_dir + '/warm_optimizer.pth'
# torch.save(warm_optimizer.state_dict(), warm_optimizer_path)

# last_layer_optimizer_path = model_dir + '/last_layer_optimizer.pth'
# torch.save(last_layer_optimizer.state_dict(), last_layer_optimizer_path)

# # Save other variables
# variables_path = model_dir + '/variables.pkl'
# variables = {
#     'coefs': coefs,
#     'num_train_epochs': num_train_epochs,
#     'num_warm_epochs': num_warm_epochs,
#     'push_start': push_start,
#     'push_epochs': push_epochs,
#     'class_specific': class_specific,
#     # Include other variables you want to save
# }
# with open(variables_path, 'wb') as f:
#     pickle.dump(variables, f)